# 2D Ising model

In this section, we move from the 1D Ising model to the 2D Ising model with the nearest-neighbor interaction.
The Hamiltonian is given by

$$
\mathcal{H} = -\sum_{\langle\langle i, j\rangle\rangle} S_i S_j,
$$

where the sum runs over pairs of nearest-neighbor sites.
The thermodynamic properties of the 2D Ising model are qualitatively different from those of the 1D model: The 2D model shows a continuous transitoin **at a finite temperature**.

In [5]:
@show VERSION
using BenchmarkTools, StaticArrays
using Random: default_rng, seed!

VERSION = v"1.5.2"


In the cell below, you can find [a very optimized code for the 2D Ising model written by Gen Kuroki](https://nbviewer.jupyter.org/gist/genkuroki/79fd71a75b46303347bed4e52aaa2ba6).
On my laptop (Macbook Pro 16-inchi 2019), using @invounds, @simd improves the performance around 7%.
But, they are dropped in the cell below.
The following code was modified from the original code to use the same update algorithm as in our 1D Ising code.

In [50]:
function ising2d_ifelse!(s, β, niters, rng=default_rng())
    m, n = size(s)
    min_h = -4
    max_h = 4
    prob = [1/(1+exp(-2*β*h)) for h in min_h:max_h]
    for iter in 1:niters
        for j in 1:n 
            for i in 1:m
                NN = s[ifelse(i == 1, m, i-1), j]
                SS = s[ifelse(i == m, 1, i+1), j]
                WW = s[i, ifelse(j == 1, n, j-1)]
                EE = s[i, ifelse(j == n, 1, j+1)]
                h = NN + SS + WW + EE
                s[i,j] = ifelse(rand(rng) < prob[h-min_h+1], +1, -1)
            end
        end
    end
end

const β_crit = log(1+sqrt(2))/2
rand_ising2d(m, n=m) = rand(Int8[-1, 1], m, n)

seed!(4649)
s₀ = rand_ising2d(100);

s = copy(s₀)
rng = default_rng()
ising2d_ifelse!(s, β_crit, 10, rng)

s = copy(s₀)
seed!(4649)
rng = default_rng()
@benchmark ising2d_ifelse!(s, β_crit, 10^3, rng)

BenchmarkTools.Trial: 
  memory estimate:  160 bytes
  allocs estimate:  1
  --------------
  minimum time:     48.798 ms (0.00% GC)
  median time:      53.439 ms (0.00% GC)
  mean time:        54.193 ms (0.00% GC)
  maximum time:     67.861 ms (0.00% GC)
  --------------
  samples:          93
  evals/sample:     1

We focus on the nearest-neighbor interaction in this section.
But, you may want to introduce further neighbor interactions in the future.
You will learn how to implement a general Monte Carlo code in this section.
We first define a struc for storing the spatial structure of the interactions.

In [57]:
module MC

struct JModel
    connected_sites::Array{Int32,2}
    # Using Array{Int8} will ruin the performance a lot! (Note missing "1"!)
    coord_num::Vector{Int8}
end

compute_effective_field(s, ispin, jmodel) = sum((s[jmodel.connected_sites[j, ispin]] for j in 1:jmodel.coord_num[ispin]))

function ising_update!(jmodel, s, β, niters, max_h, rng)
    num_spins = length(s)
    min_h = -max_h
    prob = [1/(1+exp(-2*β*h)) for h in min_h:max_h]
    h = 0
    for iter in 1:niters
        for ispin in 1:num_spins 
            h = compute_effective_field(s, ispin, jmodel)
            s[ispin] = ifelse(rand(rng) < prob[h-min_h+1], +1, -1)
        end
    end
end

end

Main.MC

Here we define a constructor for the nearest-neighbor 2D model.

In [53]:
"""
Constructor for a nearest-neighbor 2D
"""
function nn_model_2D(L)
    sites = Array{Int32,2}(undef, 4, L^2)
    mymod(i) = mod(i+L-1, L) + 1
    to_site_idx(i,j) = mymod(i) + L*(mymod(j)-1)
    disp = [[-1, 0], [1, 0], [0, -1], [0, 1]]
    for j in 1:L
        for i in 1:L
            isite = to_site_idx(i, j)
            for d in 1:4
               sites[d, isite] = to_site_idx(i+disp[d][1], j+disp[d][2])
            end
        end
    end
    MC.JModel(sites, fill(Int8(4), L^2))
end

nn_model_2D

In [56]:
L = 100
num_spins = L^2

model = nn_model_2D(L)
s = copy(vcat(s₀...))
seed!(4649)
rng = default_rng()
@benchmark MC.ising_update!(model, s, β_crit, 10^3, 4, rng)

BenchmarkTools.Trial: 
  memory estimate:  160 bytes
  allocs estimate:  1
  --------------
  minimum time:     63.241 ms (0.00% GC)
  median time:      67.425 ms (0.00% GC)
  mean time:        67.932 ms (0.00% GC)
  maximum time:     75.597 ms (0.00% GC)
  --------------
  samples:          74
  evals/sample:     1

This general code runs slower than the original code by around 25% because the structure of the interactions is not hardcoded.